In [ ]:
import psycopg2

DATABASE_CONFIG = {
    "dbname": "tensorflow_data",
    "user": "postgres",
    "password": "",
    "host": "",
    "port": 5432,
}

conn = psycopg2.connect(**DATABASE_CONFIG)
cursor = conn.cursor()

In [2]:
cursor.execute(
    """
    SELECT 
        issue_id,
        body
    FROM 
        issues_from_release
    WHERE 
        release_number LIKE '%2.12%';
    """
)
issues = cursor.fetchall()
len(issues)

305

In [3]:
import spacy

# Carrega o modelo em inglês (certifique-se de instalá-lo com: python -m spacy download en_core_web_sm)
nlp = spacy.load("en_core_web_sm")

In [ ]:
num_issues = len(issues)
num_tokens = 0
num_comments = 0
for issue in issues:
    issue_id, body = issue
    
    # Processa com o tokenizer
    try:
        doc_issue = nlp(body)
        tokens_issue = [token.text for token in doc_issue if not token.is_space and not token.is_punct]
        num_tokens += len(tokens_issue)
    except:
        pass

    cursor.execute(
        """
        SELECT 
            comment_id,
            body
        FROM 
            issue_comments_from_release
        WHERE 
            issue_id = %s;
        """, (issue_id,)
    )   
    comments = cursor.fetchall()

    num_comments += len(comments)

    for comment in comments:
        comment_id, comment_body = comment
        
        # Processa o texto com o modelo de tokenizacao
        doc = nlp(comment_body)
        # Extrai os tokens ignorando espaços e pontuação
        tokens = [token.text for token in doc if not token.is_space and not token.is_punct]

        num_tokens += len(tokens)

print(f"Number of issues: {num_issues}")
print(f"Number of comments: {num_comments}")
print(f"Number of tokens: {num_tokens}")

Number of issues: 305
Number of comments: 2442
Number of tokens: 182134


In [17]:
cursor.close()
conn.close()